In [1]:
# Allow imports from another folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np

# https://github.com/kamperh/lecture_dtw_notebook/blob/main/dtw.ipynb


def dtw(x, y, cost):
    N = x.shape[0]
    M = y.shape[0]
    dist_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = cost(x[i], y[j])

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    # return (path[::-1], cost_mat)

    return cost_mat[N - 1, M - 1], cost_mat[N - 1, M - 1] / (N + M)

In [3]:
from structs.functions import cosine
from structs.types import Frame


# creates a cost function which indexes the correct joint
def cost_function_generator(landmark_idx):
    def cost(frame1: Frame, frame2: Frame):
        frame1_point = frame1.pose_landmarks[landmark_idx]
        frame2_point = frame2.pose_landmarks[landmark_idx]
        return cosine([frame1_point.x, frame1_point.y], [frame2_point.x, frame2_point.y])

    return cost

In [4]:
def compareMotions(series1, series2):
    delta_length = len(series2) - len(series1)
    if delta_length < 0:
        delta_length = abs(delta_length)
        series1 = series1[delta_length // 2: -delta_length // 2]
    elif delta_length > 0:
        delta_length = abs(delta_length)
        series2 = series2[delta_length // 2: -delta_length // 2]

    results = []
    for i in range(11, 17):
        results.append(dtw(np.array(series1), np.array(series2), cost_function_generator(i))[1])

    return results

In [ ]:
def compareMotionsSegmented(series1, series2):
    delta_length = len(series2) - len(series1)
    if delta_length < 0:
        delta_length = abs(delta_length)
        series1 = series1[delta_length // 2: -delta_length // 2]
    elif delta_length > 0:
        delta_length = abs(delta_length)
        series2 = series2[delta_length // 2: -delta_length // 2]

    segments = 6
    interval = len(series1) // segments

    overall_result = [0] * 6
    for i in range(segments):
        segment1 = series1[i * interval: (i + 1) * interval]
        segment2 = series2[i * interval: (i + 1) * interval]
        segment_results = compareMotions(segment1, segment2)

        for key, result in enumerate(segment_results):
            overall_result[key] = result

    return overall_result

In [5]:
# Sample Usage
if __name__ == "__main__":
    import pickle

    data = []

    with open("../dataset/gestures/0.pkl", "rb") as reader:
        data.append(pickle.load(reader))

    with open("../dataset/gestures/6.pkl", "rb") as reader:
        data.append(pickle.load(reader))

    print(compareMotions(data[0].clips[0].frames, data[0].clips[1].frames))
    print(compareMotions(data[0].clips[0].frames, data[1].clips[0].frames))

    print(compareMotionsSegmented(data[0].clips[0].frames, data[0].clips[1].frames))
    print(compareMotionsSegmented(data[0].clips[0].frames, data[1].clips[0].frames))

[np.float64(0.5019937504593358), np.float64(0.48877732048602485), np.float64(0.5018875053344033), np.float64(0.48956091373530314), np.float64(0.5018334371629652), np.float64(0.4335386301293349)]
[np.float64(0.5020007275906272), np.float64(0.49629914550342824), np.float64(0.5019749144028722), np.float64(0.4988703108570067), np.float64(0.5012235857847652), np.float64(0.47950788045866605)]
